# Project Phase 1

In [1]:
import requests
from bs4 import BeautifulSoup as clean
import bs4
import pandas as pd
import re
import os
import threading

## Getting Data
This secton of code gives a list of all available plots present in Lahore.<br>
We can then select specific areas to fetch data of all the plots in that area only

In [2]:
# Getting main page to extract locations
base_url = 'https://www.zameen.com'
page = requests.get('https://www.zameen.com/Plots/Lahore-1-1.html')
beautified = clean(page.text)

In [3]:
# Get all avaialble areas
res_set = beautified.find_all('a', attrs={'class': '_2dd261fc'})

### Getting Links for Plots
##### Select Location
This section of the code displays all the available plot locations for selection and getting base_url

In [39]:
locations = [(re.sub(r'\(.*\)', '', x.text), x.get('href')) for x in res_set]
for i in range(len(locations)): print(i + 1, locations[i][0], sep=' - ')
selection = None
try:
    selection = int(input('Type number of the location you want to extract plot data from'))
except ValueError:
    print('Input should be integer')
location_name = locations[selection - 1][0]
fetch_url = base_url + locations[selection - 1][1]
fetch_url = fetch_url.rsplit(sep='-', maxsplit=1)[0]

1 - DHA Defence
2 - Raiwind Road
3 - Bahria Town
4 - Bahria Orchard
5 - GT Road
6 - Central Park Housing Scheme
7 - Park View City
8 - LDA Avenue
9 - New Lahore City
10 - DHA 11 Rahbar
11 - Bedian Road
12 - Khayaban-e-Amin
13 - LDA Road
14 - Fazaia Housing Scheme
15 - Al Rehman Garden
16 - Cantt
17 - Paragon City
18 - State Life Housing Society
19 - Sui Gas Housing Society
20 - Lahore Motorway City
21 - Canal Garden
22 - Safari Garden Housing Scheme
23 - IEP Engineers Town
24 - Park Avenue Housing Scheme
25 - Formanites Housing Scheme
26 - Johar Town
27 - Ferozepur Road
28 - Lahore - Jaranwala Road
29 - Grand Avenues Housing Scheme
30 - Chinar Bagh
31 - NFC 2
32 - Defence Road
33 - Jubilee Town
34 - Bahria Education & Medical City
35 - Eden
36 - DHA City
37 - Wapda Town
38 - OPF Housing Scheme
39 - Main Canal Bank Road
40 - Bankers Avenue Cooperative Housing Society
41 - Pak Arab Housing Society
42 - Valencia Housing Society
43 - EME Society
44 - Bahria Nasheman
45 - Al-Raziq Garden
46

##### Get Links
Get the links of all plot listings available on all pages

In [40]:
page_number = 1
links = []

while True:
    # Opening selected page
    page = requests.get(fetch_url + '-' + str(page_number) + '.html')
    beautified = clean(page.text)
    page_number += 1
    
    # Listings on this page
    listings = beautified.find_all('a', class_='_7ac32433')
    if len(listings) == 0: break
    this_page = [x.get('href') for x in listings]
    links.append(this_page)
links = sum(links, [])

##### Creating CSV for links
Creating csv for all the links to reduce overhead of computation

In [41]:
links_df = pd.DataFrame(links, columns=['links'])
links_df.to_csv(location_name + '.csv')

##### Combined Above Code
L191011 - Hasaan

In [52]:
locations = [(re.sub(r'\(.*\)', '', x.text), x.get('href')) for x in res_set]

for location_name, location_url in locations[20:39]:
    fetch_url = base_url + location_url
    fetch_url = fetch_url.rsplit(sep='-', maxsplit=1)[0]
    
    # Fetch all urls in location
    page_number = 1
    links = []

    while True:
        # Opening selected page
        page = requests.get(fetch_url + '-' + str(page_number) + '.html')
        beautified = clean(page.text)
        page_number += 1

        # Listings on this page
        listings = beautified.find_all('a', class_='_7ac32433')
        if len(listings) == 0: break
        this_page = [x.get('href') for x in listings]
        links.append(this_page)
    links = sum(links, [])
    
    #Output fetched urls in csv
    links_df = pd.DataFrame(links, columns=['links'])
    links_df.to_csv(location_name + '.csv')
    
    # Print name of completed csv
    print(f'Links from {location_name} output to csv!')

Links from Safari Garden Housing Scheme output to csv!
Links from Canal Garden output to csv!
Links from IEP Engineers Town output to csv!
Links from Park Avenue Housing Scheme output to csv!
Links from Formanites Housing Scheme output to csv!
Links from Chinar Bagh output to csv!
Links from Johar Town output to csv!
Links from Ferozepur Road output to csv!
Links from Grand Avenues Housing Scheme output to csv!
Links from Defence Road output to csv!
Links from NFC 2 output to csv!
Links from Lahore - Jaranwala Road output to csv!
Links from Jubilee Town output to csv!
Links from Bahria Education & Medical City output to csv!
Links from Eden output to csv!
Links from Wapda Town output to csv!
Links from DHA City output to csv!
Links from OPF Housing Scheme output to csv!
Links from Main Canal Bank Road output to csv!


### Scraping Data from Links
Each csv file is opened in this loop and read into a list which is then appended to the base URL to fetch the actual listings page.<br>
The target is to get <b>Ammenities</b>, <b>Trend graph</b>, and <b>Plot price</b> to form a data collection that can be used for prediction

In [8]:
import time # Import for checking execution time for each file process
from datetime import datetime

# Function to get plot details
# Class names passed as a parameter in case of future updates
def get_plot_details(page, class_='_812aa185'):
    details = page.find_all('span', attrs={'class': class_})
    return {detail.get('aria-label'): detail.text.strip() for detail in details}
def get_plot_amenities(page, class_ = '_17984a2c'):
    amenities = page.find_all('span', attrs={'class': class_})
    return ','.join([amenity.text.strip() for amenity in amenities])

# Getting details done in a function to promote parallel code.
def get_details(file):
    links = pd.read_csv('Links/' + file) # Links in files read into links
    
    # Check execution time code
    start_time_s = time.time()
    print('Time code execution began: ', datetime.now().strftime('%H:%M:%S'))
    # End of execution time code
    
    print(f'Started on Listings in file: {file}', f'File Length: {len(links)}', sep='\n', end='\n\n')
    push_list = [] # List for storing all dictionaries to be added in csv file
    # Secondary loop to iterate over all links and get data
    for _, link in links.values:
        current_page = clean(requests.get(base_url + link).text) # Get page
        push_dict = get_plot_details(current_page) # Get plot details
        push_dict['Amenities'] = get_plot_amenities(current_page) # Get Amenities
        push_list.append(push_dict) # Append dictionary to list
    
    # Check execution time code
    total_time = round(time.time() - start_time_s, 5)
    print('Time code execution ended: ', datetime.now().strftime('%H:%M:%S'))
    # End of execution time code
    
    file_name = file.split(sep='.')[0]
    pd.DataFrame.from_dict(push_list).to_csv(file_name + '_data.csv') # Output in dataframe
    print(f'Data for all links in {file} output!', f'Time taken {total_time}', sep='\n', end='\n\t-----\n')
    
# Same function above with progress percentage
def _get_details(file):
    links = pd.read_csv('Links/' + file) # Links in files read into links
    
    # Check execution time code
    start_time_s = time.time()
    print('Time code execution began: ', datetime.now().strftime('%H:%M:%S'))
    # End of execution time code
    
    print(f'Started on Listings in file: {file}', f'File Length: {links.shape[0]}', sep='\n')
    push_list = [] # List for storing all dictionaries to be added in csv file
    
    itr, t_itr = 0, links.shape[0]#Progress update
    # Secondary loop to iterate over all links and get data
    for _, link in links.values:
        current_page = clean(requests.get(base_url + link).text) # Get page
        push_dict = get_plot_details(current_page) # Get plot details
        push_dict['Amenities'] = get_plot_amenities(current_page) # Get Amenities
        push_list.append(push_dict) # Append dictionary to list
        itr += 1
        print(f'{"{:.2f}".format((itr/t_itr)*100)}%', end='\r') # Output percentage
    
    # Check execution time code
    total_time = round(time.time() - start_time_s, 5)
    print('Time code execution ended: ', datetime.now().strftime('%H:%M:%S'))
    # End of execution time code
    
    file_name = file.split(sep='.')[0]
    pd.DataFrame.from_dict(push_list).to_csv(file_name + '_data.csv') # Output in dataframe
    print(f'Data for all links in {file} output!', f'Time taken {total_time}', sep='\n', end='\n\t-----\n')

In [10]:
# Main loop for data collection
links_file_names = list(os.listdir('Links')) # Get all file names for csv files with links
# The below sorting is done so equal size threads execute at once to reduce wastage
links_file_names = sorted(links_file_names, key=lambda x: os.stat('Links/' + x).st_size) # Sort files by size

# Main loop iterates over all file names
# Threaded version
"""
for file_i in range(0,len(links_file_names),4):
    threads = []
    for j in range(4):
        T = threading.Thread(target=get_details, args=(links_file_names[file_i + j],), daemon=True)
        threads.append(T)
        T.start()
    for _, thread in enumerate(threads):
        thread.join()
"""
print(links_file_names)
# Non Threaded version
for file in links_file_names:
    _get_details(file)

['DHA Defence.csv']
Time code execution began:  06:13:14
Started on Listings in file: DHA Defence.csv
File Length: 13869
Time code execution ended:  09:46:07
Data for all links in DHA Defence.csv output!
Time taken 12773.66685
	-----


###### Threading Help
import threading
import time

def thread_function(name):
    print(f'Started thread {name}')
    time.sleep(name)
    print('Thread Done', name, flush=True)

if __name__ == "__main__":

    threads = list()
    for index in range(3):
        x = threading.Thread(target=thread_function, args=(index,) ,  daemon=True)
        threads.append(x)
        x.start()

    for index, thread in enumerate(threads):
        thread.join()

## Cleaning Data (Data Wrangling)
The following portion first reads all the csv files into a single dataframe and appends a new column that shows the location of the plot. The following part cleans and applies a few basic data wrangling principles to improve the visualisation of data.
### Reading into DataFrame
All csv files are read into a single DataFrame

In [25]:
file_names = os.listdir('Data')
df_plots = pd.read_csv('Data/' + file_names[0]) # DataFrame created

for file in file_names[1:]:
    df_temp = pd.read_csv('Data/' + file)
    df_plots = pd.concat([df_plots, df_temp])
    del df_temp
df_plots.reset_index(drop=True, inplace=True)
del file_names
del file
df_plots

,Unnamed: 0,Type,Price,Location,Baths,Area,Purpose,Beds,Creation date,Amenities,advance,Monthly installment,Remaining installments
0,0,Residential Plot,PKR19 Lakh,"Al Jalil Garden, Lahore, Punjab",-,5 Marla,For Sale,-,1 day ago,"Possesion,File,Balloted,Sewerage,Electricity,W...",NaN,NaN,NaN
1,1,Residential Plot,PKR45 Lakh,"Al Jalil Garden, Lahore, Punjab",-,5 Marla,For Sale,-,1 day ago,"Possesion,File,Balloted,Sewerage,Electricity,W...",NaN,NaN,NaN
2,2,Residential Plot,PKR59 Lakh,"Al Jalil Garden, Lahore, Punjab",-,7.5 Marla,For Sale,-,4 days ago,NaN,NaN,NaN,NaN
3,3,Residential Plot,PKR59 Lakh,"Al Jalil Garden, Lahore, Punjab",-,7.5 Marla,For Sale,-,4 days ago,NaN,NaN,NaN,NaN
4,4,Residential Plot,PKR59 Lakh,"Al Jalil Garden, Lahore, Punjab",-,7.5 Marla,For Sale,-,4 days ago,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
44026,34,Residential Plot,PKR21 Lakh,"Zaitoon City, Lahore, Punjab",-,3 Marla,For Sale,-,5 months ago,"Possesion,Corner,Park Facing,Disputed,File,Bal...",PKR4.75 Lakh,PKR17 Thousand,33 Months
44027,35,Residential Plot,PKR22.5 Lakh,"Zaitoon City, Lahore, Punjab",-,4.4 Marla,For Sale,-,5 months ago,NaN,NaN,NaN,NaN
44028,36,Plot File,PKR2.25 Lakh,"Zaitoon City, Lahore, Punjab",-,3 Marla,For Sale,-,6 months ago,NaN,NaN,NaN,NaN
44029,37,Plot File,PKR1.5 Lakh,"Zaitoon City, Lahore, Punjab",-,5 Marla,For Sale,-,6 months ago,NaN,NaN,NaN,NaN


### Cleaning Data
Unnecessary columns are dropped

In [26]:
df_plots.drop(columns=['Unnamed: 0', 'Beds', 'Baths', 'Purpose'], axis=1, inplace=True)
drop_list = list(df_plots[df_plots['Price'].isnull()].index)
df_plots.drop(drop_list, inplace=True)
del drop_list
df_plots.reset_index(drop=True, inplace=True)
df_plots

,Type,Price,Location,Area,Creation date,Amenities,advance,Monthly installment,Remaining installments
0,Residential Plot,PKR19 Lakh,"Al Jalil Garden, Lahore, Punjab",5 Marla,1 day ago,"Possesion,File,Balloted,Sewerage,Electricity,W...",NaN,NaN,NaN
1,Residential Plot,PKR45 Lakh,"Al Jalil Garden, Lahore, Punjab",5 Marla,1 day ago,"Possesion,File,Balloted,Sewerage,Electricity,W...",NaN,NaN,NaN
2,Residential Plot,PKR59 Lakh,"Al Jalil Garden, Lahore, Punjab",7.5 Marla,4 days ago,NaN,NaN,NaN,NaN
3,Residential Plot,PKR59 Lakh,"Al Jalil Garden, Lahore, Punjab",7.5 Marla,4 days ago,NaN,NaN,NaN,NaN
4,Residential Plot,PKR59 Lakh,"Al Jalil Garden, Lahore, Punjab",7.5 Marla,4 days ago,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
43315,Residential Plot,PKR21 Lakh,"Zaitoon City, Lahore, Punjab",3 Marla,5 months ago,"Possesion,Corner,Park Facing,Disputed,File,Bal...",PKR4.75 Lakh,PKR17 Thousand,33 Months
43316,Residential Plot,PKR22.5 Lakh,"Zaitoon City, Lahore, Punjab",4.4 Marla,5 months ago,NaN,NaN,NaN,NaN
43317,Plot File,PKR2.25 Lakh,"Zaitoon City, Lahore, Punjab",3 Marla,6 months ago,NaN,NaN,NaN,NaN
43318,Plot File,PKR1.5 Lakh,"Zaitoon City, Lahore, Punjab",5 Marla,6 months ago,NaN,NaN,NaN,NaN


Row-wise operations done for each column

In [27]:
for i in range(df_plots.shape[0]):
    # Sort Amenities so they can be categorically encoded
    df_plots['Amenities'].iloc[i] = ','.join(sorted(str(df_plots['Amenities'].iloc[i]).split(sep=',')))
    # Since location is in lahore, punjab we remove that part of the location
    df_plots['Location'].iloc[i] = df_plots['Location'].iloc[i].split(sep=',')[0]
    
    # Set prices in standard 1rs. unit
    if re.search(r'PKR(\d*\.*\d*) (.*)', df_plots['Price'].iloc[i]).group(2) == 'Lakh':
        df_plots['Price'].iloc[i] = int(float(re.search(r'PKR(\d*\.*\d*) (.*)', df_plots['Price'].iloc[i]).group(1)) * 100000)
    else:
        df_plots['Price'].iloc[i] = int(float(re.search(r'PKR(\d*\.*\d*) (.*)', df_plots['Price'].iloc[i]).group(1)) * 10000000)
    
    # Set plot in standard marlas unit
    if re.search(r'(\d*\.*\d*) (.*)', df_plots['Area'].iloc[i]).group(2) == 'Marla':
        df_plots['Area'].iloc[i] = float(re.search(r'(\d*\.*\d*) (.*)', df_plots['Area'].iloc[i]).group(1))
    else:
        df_plots['Area'].iloc[i] = float(re.search(r'(\d*\.*\d*) (.*)', df_plots['Area'].iloc[i]).group(1)) * 20

# Convert both the data types to numeric
df_plots['Area'] = pd.to_numeric(df_plots['Area'])
df_plots['Price'] = pd.to_numeric(df_plots['Price'])

Categorically encoded Amenities

In [28]:
from pandas.api.types import CategoricalDtype

categories = CategoricalDtype(list(df_plots['Amenities'].unique()), ordered=True)
df_plots['Amenities'] = df_plots['Amenities'].astype(categories).cat.codes

### Standardising Prices